In [1]:
import os
import numpy as np
from PIL import Image
import random
import math
import pandas as pd
import cv2
import pandas as pd
from tqdm import tqdm

In [59]:
# 1. Carica l’immagine e normalizza in [0,1]
img = Image.open('cat.jpg').convert('RGB')
img_arr = np.asarray(img, dtype=np.float32) / 255.0


b = 0.17
a = -b
noisy_arr = add_salt_pepper_noise(img_arr)
noisy_img = Image.fromarray((noisy_arr * 255).astype(np.uint8))
# 4a. Salva su disco
#noisy_img.save('noisy_image.jpg')

# 4b. Oppure mostra a video (apre il viewer predefinito)
noisy_img.show()
#Andiamo

In [ ]:
def add_salt_pepper_noise(image: np.ndarray, amount: float = 0.05, salt_vs_pepper: float = 0.5) -> np.ndarray:
    """
    :param image: Input image as float32 numpy array in [0,1], shape (H, W, 3).
    :param amount: Fraction of total pixels to corrupt.
    :param salt_vs_pepper: Fraction of corrupted pixels that are salt (vs pepper).
    :return: Noisy image as numpy array.
    """
    noisy = np.copy(image)
    ROWS, COLS, _ = image.shape
    num_pixels = ROWS * COLS
    num_salt = int(np.ceil(amount * num_pixels * salt_vs_pepper))
    num_pepper = int(np.ceil(amount * num_pixels * (1 - salt_vs_pepper)))

    # SALT
    coords_y = np.random.randint(0, ROWS, num_salt)
    coords_x = np.random.randint(0, COLS, num_salt)
    noisy[coords_y, coords_x, :] = 1.0

    # PEPPER
    coords_y = np.random.randint(0, ROWS, num_pepper)
    coords_x = np.random.randint(0, COLS, num_pepper)
    noisy[coords_y, coords_x, :] = 0.0

    return noisy

rng = np.random.default_rng()


def add_gaussian_noise(image: np.ndarray, mean=0.0, var=0.01):
    """
    :param image: Input image as a float32 numpy array with values in [0, 1].
    :param mean: Mean of Gaussian noise.
    :param var: Variance of Gaussian noise.
    :return: Noisy image as numpy array.
    """
    sigma = np.float32(var**0.5)
    noisy = image.copy()  # float32
    rng.standard_normal(size=image.shape, dtype=np.float32, out=noisy)
    noisy *= sigma
    noisy += image + np.float32(mean)
    np.clip(noisy, 0.0, 1.0, out=noisy)
    return noisy


def add_uniform_noise(image: np.ndarray, low=-0.1732, high=0.1732):
    """
    :param image: Input image as a float32 numpy array with values in [0, 1].
    :param low: Lower bound of uniform noise.
    :param high: Upper bound of uniform noise.
    :return: Noisy image as numpy array.
    """
    noisy = image.copy()  # float32
    rng.random(size=image.shape, dtype=np.float32, out=noisy)
    noisy *= np.float32(high - low)
    noisy += np.float32(low) + image
    np.clip(noisy, 0.0, 1.0, out=noisy)
    return noisy



def add_erlang_noise(image: np.ndarray, k=3, lam=17.32):
    """
    Add Erlang (Gamma) noise to an image, using shape k and rate λ.
    The Erlang PDF is: f(x) = (λ^k * x^(k-1) / (k-1)!) * e^{-λ x},  x >= 0

    :param image:     Input image as float32 numpy array in [0,1], shape (H, W, C).
    :param k:         Shape parameter (integer ≥ 1).
    :param lam:       Rate parameter λ (> 0).
    :return:          Noisy image as numpy array, clipped to [0,1].
    """
    theta = np.float32(1.0/lam)
    noisy = image.copy() # float32
    noise64 = rng.gamma(shape=k, scale=theta, size=image.shape) # genero rumore in float64 (per forza)
    noise = noise64.astype(np.float32) # cast to float32 (necessario)
    noise -= (np.float32(k) * theta)
    noisy += noise
    np.clip(noisy, 0.0, 1.0, out=noisy)
    return noisy


def add_striping_noise(image: np.ndarray, amplitude: float = 0.5, frequency: float = 50, orientation: str = 'vertical') -> np.ndarray:
    """
    Aggiunge rumore a bande (striping noise) a un'immagine a colori, 
    con la possibilità di generare bande verticali o orizzontali.

    :param image:       Immagine di input come array numpy float32 con valori in [0,1], shape (H, W, 3).
    :param amplitude:   Ampiezza del pattern sinusoidale (massimo valore del rumore).
    :param frequency:   Frequenza del pattern sinusoidale (numero di cicli completi su tutta la dimensione scelta).
    :param orientation: 'vertical' per bande verticali, 'horizontal' per bande orizzontali.
    :return:             Immagine rumorosa come array numpy float32, ritagliata in [0,1].
    """
    # Validazione dell'orientamento
    if orientation not in ('vertical', 'horizontal'):
        raise ValueError("orientation deve essere 'vertical' oppure 'horizontal'")

    # Copia dell'immagine di input
    noisy = image.copy().astype(np.float32)

    # Dimensioni
    H, W, C = image.shape

    if orientation == 'vertical':
        # Bande verticali: variazione secondo l'indice di colonna (x)
        x = np.arange(W, dtype=np.float32)
        # Pattern sinusoidale 1D lungo l'asse orizzontale, con cicli su tutta la larghezza W
        stripe_1d = amplitude * np.sin(2.0 * np.pi * frequency * x / W).astype(np.float32)
        # Espando a 2D: stessa riga ripetuta per H righe
        stripe_2d = np.tile(stripe_1d[np.newaxis, :], (H, 1))

    else:  # orientation == 'horizontal'
        # Bande orizzontali: variazione secondo l'indice di riga (y)
        y = np.arange(H, dtype=np.float32)
        # Pattern sinusoidale 1D lungo l'asse verticale, con cicli su tutta l'altezza H
        stripe_1d = amplitude * np.sin(2.0 * np.pi * frequency * y / H).astype(np.float32)
        # Espando a 2D: stessa colonna ripetuta per W colonne
        stripe_2d = np.tile(stripe_1d[:, np.newaxis], (1, W))

    # Aggiungo il rumore a bande a ciascun canale di colore
    # stripe_2d ha shape (H, W), lo espandiamo a (H, W, 1) e si somma via broadcasting
    noisy += stripe_2d[:, :, np.newaxis]

    # Ritaglio i valori in [0,1]
    np.clip(noisy, 0.0, 1.0, out=noisy)

    return noisy

In [ ]:
# 1. Carica e normalizza
img = Image.open('cat.jpg').convert('RGB')
arr = np.asarray(img, dtype=np.float32) / 255.0

noisy_arr = add_erlang_noise(arr, k=3, lam=25)
# 12,25
# 3. Riconverti e salva
noisy_img = Image.fromarray((noisy_arr * 255).astype(np.uint8))
noisy_img.show()
#noisy_img.save('path/to/noisy_erlang.jpg')

Bisogna calcolare la varianza per ogni rumore, e settare i parametri affinchè la varianza media sia 0.01.

In [3]:
def generate_dataset(input_dir: str, output_dir: str, batch_size: int = 32) -> None:
    """
    Genera un dataset rumoroso con:
      - originali
      - salt & pepper
      - gaussian
      - uniform
      - erlang
    usando cv2 per I/O veloce e batch-loading,
    e una barra di progresso su tutte le immagini con tqdm.
    """
    # Range parametri
    sp_range     = (0.01, 0.10)
    gauss_range  = (0.005, 0.02)
    uni_range    = (0.10, 0.20)
    erlang_range = (12.0, 25.0)
    k_erlang     = 3

    variants = ['original','salt_pepper','gaussian','uniform','erlang']
    label_map = {
        'original':    'Original',
        'salt_pepper': 'Salt & Pepper',
        'gaussian':    'Gaussian',
        'uniform':     'Uniform',
        'erlang':      'Erlang'
    }
    # Crea cartelle di output
    for v in variants:
        os.makedirs(os.path.join(output_dir, v), exist_ok=True)

    # Elenca tutti i file immagine
    files = [f for f in os.listdir(input_dir)
             if f.lower().endswith(('.png','.jpg','.jpeg','.bmp','.tiff'))]

    records = []
    # Inizializza tqdm su numero totale di immagini
    pbar = tqdm(total=len(files), desc="Processing images")

    # Elaborazione a batch
    for i in range(0, len(files), batch_size):
        batch = files[i:i+batch_size]
        imgs = []
        # 1) carica batch
        for fn in batch:
            src = os.path.join(input_dir, fn)
            bgr = cv2.imread(src, cv2.IMREAD_COLOR)
            rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
            imgs.append(rgb.astype(np.float32) / 255.0)

        # 2) per ciascuna immagine del batch, applica rumori e salva
        for idx, fn in enumerate(batch):
            img = imgs[idx]

            # — Originale —
            out_orig = os.path.join(output_dir, 'original', fn)
            cv2.imwrite(out_orig,
                        cv2.cvtColor((img*255).astype(np.uint8), cv2.COLOR_RGB2BGR))
            records.append({'filepath': out_orig, 'label': label_map['original']})

            # — Salt & Pepper —
            amt = random.uniform(*sp_range)
            sp = add_salt_pepper_noise(img, amount=amt)
            out_sp = os.path.join(output_dir, 'salt_pepper', fn)
            cv2.imwrite(out_sp,
                        cv2.cvtColor((sp*255).astype(np.uint8), cv2.COLOR_RGB2BGR))
            records.append({'filepath': out_sp, 'label': label_map['salt_pepper']})

            # — Gaussian —
            var = random.uniform(*gauss_range)
            g = add_gaussian_noise(img, var=var)
            out_g = os.path.join(output_dir, 'gaussian', fn)
            cv2.imwrite(out_g,
                        cv2.cvtColor((g*255).astype(np.uint8), cv2.COLOR_RGB2BGR))
            records.append({'filepath': out_g, 'label': label_map['gaussian']})

            # — Uniform —
            r = random.uniform(*uni_range)
            u = add_uniform_noise(img, low=-r, high=+r)
            out_u = os.path.join(output_dir, 'uniform', fn)
            cv2.imwrite(out_u,
                        cv2.cvtColor((u*255).astype(np.uint8), cv2.COLOR_RGB2BGR))
            records.append({'filepath': out_u, 'label': label_map['uniform']})

            # — Erlang —
            lam = random.uniform(*erlang_range)
            e = add_erlang_noise(img, k=k_erlang, lam=lam)
            out_e = os.path.join(output_dir, 'erlang', fn)
            cv2.imwrite(out_e,
                        cv2.cvtColor((e*255).astype(np.uint8), cv2.COLOR_RGB2BGR))
            records.append({'filepath': out_e, 'label': label_map['erlang']})

            # aggiorna la barra di progresso
            pbar.update(1)

    pbar.close()

    # Salva CSV delle label
    pd.DataFrame(records).to_csv(os.path.join(output_dir, 'labels.csv'), index=False)
    print(f"Dataset e labels salvati in '{output_dir}'")


In [ ]:
# ESEGUO
input_directory = "CocoMiniMini_2k/."
output_directory = "Dataset_2000x5."
generate_dataset(input_directory, output_directory)

Processing images: 100%|██████████| 2000/2000 [01:56<00:00, 17.22it/s]

Dataset e labels salvati in 'Dataset_2000x5.'


In [4]:
import time

# Benchmark
img = Image.open('cat.jpg').convert('RGB')
image = np.asarray(img, dtype=np.float32) / 255.0
functions = {
    'Salt & Pepper': lambda img: add_salt_pepper_noise(img),
    'Gaussian': lambda img: add_gaussian_noise(img),
    'Uniform': lambda img: add_uniform_noise(img),
    'Erlang (Gamma)': lambda img: add_erlang_noise(img)
}

N = 60
results = {}
for name, fn in functions.items():
    start = time.time()
    for _ in range(N):
        _ = fn(image)
    end = time.time()
    results[name] = end - start

for name, total in results.items():
    print(f"{name}: {total:.4f}s total, {total/N:.6f}s per run")


Salt & Pepper: 0.2359s total, 0.003932s per run
Gaussian: 1.3666s total, 0.022776s per run
Uniform: 0.6268s total, 0.010446s per run
Erlang (Gamma): 2.9143s total, 0.048572s per run
